In [19]:
!pip install -q efficientnet kagglehub

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import cv2
import os
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
import efficientnet.tfkeras as efn


In [20]:
path = kagglehub.dataset_download("harunshimanto/epileptic-seizure-recognition")
csv_path = os.path.join(path, "Epileptic Seizure Recognition.csv")
df = pd.read_csv(csv_path)
df.drop(columns=[df.columns[0]], inplace=True)
df = df.drop(columns=['Unnamed'], errors='ignore')
df = df[df.iloc[:, 1:179].sum(axis=1) != 0]

X = df.iloc[:, 1:179].values
y = df.iloc[:, -1].values - 1  

In [21]:
def signal_to_stft_image(signal, sr=178):
    signal = np.asarray(signal, dtype=np.float32)
    stft = librosa.stft(signal, n_fft=64)
    stft_db = librosa.amplitude_to_db(np.abs(stft))
    stft_db_resized = cv2.resize(stft_db, (128, 128))
    return stft_db_resized

X_images = np.array([signal_to_stft_image(x) for x in X])
X_images = np.expand_dims(X_images, axis=-1)
X_images = np.repeat(X_images, 3, axis=-1)  
X_images = X_images / 255.0  


X_train, X_test, y_train, y_test = train_test_split(
    X_images, y, test_size=0.3, stratify=y, random_state=42
)


class_weights = dict(enumerate(compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)))


In [22]:
base_model = efn.EfficientNetB0(include_top=False, input_shape=(128, 128, 3), weights=None)
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(5, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_lr=1e-6, verbose=1)
]


In [23]:
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=32,
    class_weight=class_weights,
    callbacks=callbacks
)


Epoch 1/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 682s 3s/step - accuracy: 0.5406 - loss: 1.0460 - val_accuracy: 0.1789 - val_loss: 2.2309 - learning_rate: 0.0010
Epoch 2/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 665s 3s/step - accuracy: 0.7102 - loss: 0.6130 - val_accuracy: 0.2857 - val_loss: 3.6941 - learning_rate: 0.0010
Epoch 3/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 613s 3s/step - accuracy: 0.7475 - loss: 0.5268 - val_accuracy: 0.6435 - val_loss: 0.8855 - learning_rate: 0.0010
Epoch 4/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 615s 3s/step - accuracy: 0.7725 - loss: 0.4881 - val_accuracy: 0.6286 - val_loss: 1.1866 - learning_rate: 0.0010
Epoch 5/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 620s 3s/step - accuracy: 0.7950 - loss: 0.4370 - val_accuracy: 0.7466 - val_loss: 0.5438 - learning_rate: 0.0010
Epoch 6/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 635s 3s/step - accuracy: 0.8033 - loss: 0.4267 - val_accuracy: 0.7602 - val_loss: 0.5345 - learning_rate: 0.0010
Epoch 7/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 617s 3s/step - accuracy: 0.8267 - loss: 0.

In [24]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy:", test_acc)


108/108 ━━━━━━━━━━━━━━━━━━━━ 77s 688ms/step - accuracy: 0.8067 - loss: 0.4953
Test Accuracy: 0.8063768148422241


In [25]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test).argmax(axis=1)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


108/108 ━━━━━━━━━━━━━━━━━━━━ 68s 593ms/step
              precision    recall  f1-score   support

           0       0.94      0.99      0.96       690
           1       0.69      0.60      0.64       690
           2       0.66      0.70      0.68       690
           3       0.94      0.83      0.88       690
           4       0.80      0.92      0.86       690

    accuracy                           0.81      3450
   macro avg       0.81      0.81      0.80      3450
weighted avg       0.81      0.81      0.80      3450

[[680  10   0   0   0]
 [ 36 413 225   1  15]
 [  6 173 481   0  30]
 [  0   0   3 574 113]
 [  0   3  16  37 634]]
